In [38]:
import pandas as pd
import numpy as np
import datetime
from tiingo import TiingoClient
from sklearn.impute import SimpleImputer
import indicoio
import seaborn as sns
import time
from textblob import TextBlob
import matplotlib as plt

In [39]:
UP = []
DOWN = []

def count_words(original_string, bag_of_words):
    split_string = original_string.lower().split()
    count = 0
    for word in bag_of_words:
        matches = split_string.count(word)
        count += matches
    ratio = count/len(split_string)
    return ratio
    
def aggregate_jsons(json_list):
    result = []
    for json in json_list:
        headline = json["title"]
        desc = json["description"]
        x = headline + " " + desc
        result.append(x)
    return result

def pipeline(df_path, ticky):
    # get the dataframe from csv
    df = pd.read_csv(df_path)
    
    # fix dates
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
    df["Start"] = df["Date"].apply(str)
    six_days = lambda start_date: start_date + datetime.timedelta(days=6)
    df["End"] = df["Date"].apply(six_days).apply(str)
    df = df.drop(columns=["Date"])
    remove_time = lambda dt: dt[0:10]
    df["Start"] = df["Start"].apply(remove_time)
    df["End"] = df["End"].apply(remove_time)
    
    # add json
    client = TiingoClient({"api_key": "a265fc4a1013923f970d16e7348195074e97fcb0"})
    query_ticker = lambda t, s, e: client.get_news(tickers=[t], startDate=s, endDate=e)
    df["JSON"] = df.apply(lambda d: query_ticker(ticky, d["Start"], d["End"]), axis=1)
    
    # create corpus
    df["corpus"] = df["JSON"].apply(aggregate_jsons)
    df["vectorized"] = df["corpus"]
    combinatric = lambda l: ''.join(l)
    df["corpus"] = df["vectorized"].apply(combinatric)
    indicoio.config.api_key = "25b83c4c388204edd2c6c11cd907e048"
    # add sentiment
    df["sentiment"] = df["corpus"].apply(lambda orig: TextBlob(orig).sentiment.polarity)
    df["sentiment_test"] = df["vectorized"].apply(lambda orig: [TextBlob(o).sentiment.polarity for o in orig]).apply(np.mean)
    df["indico_sentiment"] = df["corpus"].apply(lambda text: indicoio.sentiment_hq(text))
    # add statistical features
    # df["features"] = df["corpus"].apply(lambda text: indicoio.text_features(text, v=2))
    
    # add bag of words ratio
    #df["bad_ratio"] = 
    #df["good_keywords"]
    
    # return the nice beautiful dataframe
    return df

In [40]:
def predictor(d, ticky):
    from sklearn.model_selection import train_test_split
    from sklearn import linear_model

    d["constvec"] = [0.7] * len(d)
    d["lastweek"] = d["Close"]
    df.lastweek = df.lastweek.shift(7) ## shift down
    #df.lastweek.drop(df.gdp.shape[0] - 1,inplace = True) ## removing the first row
    X = d[["Open", "lastweek", "indico_sentiment"]]
    Y = d["Close"]
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp = imp.fit(X)
    X = imp.transform(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 69)

    def mse(v1, v2):
        return np.sum((v1 - v2) ** 2) 
    
    model = linear_model.BayesianRidge()
    model.fit(X_train, Y_train)
    Y_predicted = model.predict(X_test)
    error = mse(Y_predicted, Y_test)
    print("aggregated error : {0}".format(error))
    print("average error by prediction for {1}: {0}".format(np.mean(Y_predicted - Y_test), ticky))

In [41]:
start = time.time()
df = pipeline('aapl-6m-weekly.csv', 'aapl')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'aapl'))
predictor(df, 'aapl')

pipeline took 34.08493709564209s for aapl
aggregated error : 158.5053123445582
average error by prediction for aapl: 0.08795332615945187


In [42]:
start = time.time()
df = pipeline('crm-6m-weekly.csv', 'crm')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'crm'))
predictor(df, 'crm')

pipeline took 30.264259099960327s for crm
aggregated error : 87.2436031751756
average error by prediction for crm: 1.8843514114534072


In [43]:
start = time.time()
df = pipeline('tsla-6m-weekly.csv', 'tsla')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'tsla'))
predictor(df, 'tsla')

pipeline took 33.41849899291992s for tsla
aggregated error : 2980.1666694736023
average error by prediction for tsla: -11.74392175573135


In [44]:
start = time.time()
df = pipeline('fb-6m-weekly.csv', 'fb')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'fb'))
predictor(df, 'fb')

pipeline took 30.157266855239868s for fb
aggregated error : 112.77374401165784
average error by prediction for fb: 0.11871369705476001


In [45]:
start = time.time()
df = pipeline('baba-6m-weekly.csv', 'baba')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'baba'))
predictor(df, 'baba')

pipeline took 34.08108711242676s for baba
aggregated error : 343.35529935324485
average error by prediction for baba: -3.3095867717986556


In [46]:
start = time.time()
df = pipeline('msft-6m-weekly.csv', 'msft')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'msft'))
predictor(df, 'msft')

pipeline took 34.297032833099365s for msft
aggregated error : 44.32900935243161
average error by prediction for msft: 0.765565859534724


In [47]:
start = time.time()
df = pipeline('nflx-6m-weekly.csv', 'nflx')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'nflx'))
predictor(df, 'nflx')

pipeline took 32.15219712257385s for nflx
aggregated error : 1579.1503591096744
average error by prediction for nflx: -8.735652954215588


In [48]:
start = time.time()
df = pipeline('amzn-6m-weekly.csv', 'amzn')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'amzn'))
predictor(df, 'amzn')

pipeline took 31.631071090698242s for amzn
aggregated error : 3559.698342212954
average error by prediction for amzn: 9.340305573201501


In [49]:
start = time.time()
df = pipeline('googl-6m-weekly.csv', 'googl')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'googl'))
predictor(df, 'googl')

pipeline took 36.64185070991516s for googl
aggregated error : 14629.32763112144
average error by prediction for googl: 27.57323368072121


In [50]:
start = time.time()
df = pipeline('wmt-6m-weekly.csv', 'wmt')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'wmt'))
predictor(df, 'wmt')

pipeline took 34.182809829711914s for wmt
aggregated error : 55.831026301367956
average error by prediction for wmt: -0.5945596814515078
